In [24]:
import pandas as pd
import numpy as np

In [26]:
x_train=pd.read_csv('/Users/zhilinyang/Desktop/work/x_train1.csv')
x_train=x_train.iloc[:,1:]
x_train.head()

,serial_number,smart_1_raw,smart_3_normalized,smart_5_normalized,smart_7_normalized,smart_9_normalized,smart_187_normalized,smart_189_normalized,smart_194_normalized,smart_197_normalized,smart_5_raw,smart_197_raw
0,Z304HVAM,74048288.0,95.0,100.0,84.0,89.0,100.0,100.0,26.0,100.0,0.0,0.0
1,S300XRA6,100065256.0,96.0,100.0,89.0,78.0,100.0,100.0,19.0,100.0,0.0,0.0
2,Z304JMSQ,75736472.0,92.0,100.0,87.0,85.0,100.0,100.0,27.0,100.0,0.0,0.0
3,Z302B0HC,242383000.0,92.0,100.0,89.0,78.0,100.0,100.0,30.0,100.0,0.0,0.0
4,Z300MG4B,44841392.0,92.0,100.0,75.0,64.0,100.0,88.0,22.0,100.0,0.0,0.0


In [27]:
x_test=pd.read_csv('/Users/zhilinyang/Desktop/work/x_test1.csv')
x_test=x_test.iloc[:,1:]
x_test.head()

,serial_number,smart_1_raw,smart_3_normalized,smart_5_normalized,smart_7_normalized,smart_9_normalized,smart_187_normalized,smart_189_normalized,smart_194_normalized,smart_197_normalized,smart_5_raw,smart_197_raw
0,W300R8FD,210428384.0,96.0,100.0,72.0,75.0,100.0,100.0,21.0,100.0,0.0,0.0
1,Z304JMTY,185917944.0,97.0,100.0,88.0,82.0,100.0,100.0,22.0,100.0,0.0,0.0
2,Z301JZLE,235594800.0,93.0,100.0,84.0,90.0,100.0,100.0,22.0,100.0,0.0,0.0
3,S300XE2H,79078632.0,100.0,100.0,90.0,76.0,100.0,100.0,20.0,100.0,0.0,0.0
4,S300Z5LH,167795152.0,97.0,100.0,79.0,76.0,100.0,100.0,27.0,100.0,0.0,0.0


In [28]:
y_train=pd.read_csv('/Users/zhilinyang/Desktop/work/y_train1.csv',header=None)
y_train=y_train.iloc[:,1:]
y_train.head()

,1
0,0
1,0
2,0
3,0
4,0


In [29]:
y_test=pd.read_csv('/Users/zhilinyang/Desktop/work/y_test1.csv',header=None)
y_test=y_test.iloc[:,1:]
y_test.head()

,1
0,0
1,0
2,1
3,0
4,0


In [30]:
from sklearn.tree import DecisionTreeClassifier
DT=DecisionTreeClassifier(criterion='entropy',min_samples_split = 20, min_samples_leaf = 7, min_impurity_decrease = 0.001)

In [31]:
len(y_train)

111159

In [32]:
DT.fit(x_train.iloc[:,1:],y_train.values)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.001, min_impurity_split=None,
            min_samples_leaf=7, min_samples_split=20,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [33]:
DT.score(x_test.iloc[:,1:],y_test.values)

0.91885698118545767

In [34]:
y_pred=DT.predict(x_test.iloc[:,1:])

In [22]:
from sklearn.metrics import confusion_matrix
def FDR(y_true, y_pred):
    mat=confusion_matrix(y_true=y_true,y_pred=y_pred)
    TP=mat[0,0]
    FN=mat[0,1]
    FDR=TP/(TP+FN)
    print(FDR)
    return FDR

In [35]:
FDR(y_test,y_pred)

0.93340951488


0.93340951487981871

In [36]:
def FAR(y_true, y_pred):
    mat=confusion_matrix(y_true=y_true,y_pred=y_pred)
    FP=mat[1,0]
    TN=mat[1,1]
    FAR=FP/(FP+TN)
    return FAR

In [37]:
FAR(y_test,y_pred)

0.48958333333333331

In [42]:
def predict_disk(testset,y_pred,disk_name_list):
    result=[]
    test_set=testset.groupby('serial_number')
    y_pred=pd.Series(y_pred)
    for disk in disk_name_list:
        sample_index=test_set.get_group(disk).index
        sample_result=y_pred.reindex(sample_index)
        sample_result=pd.Series(sample_result)
        if sample_result.any():
            result.append(1)
        else:
            result.append(0)
    return result

In [43]:
def disk_state(testset,y_true,disk_name_list):
    result=[]
    test_set=testset.groupby('serial_number')
    for disk in disk_name_list:
        sample_index=test_set.get_group(disk).index
        sample_result=y_true.reindex(sample_index)
        sample_result=sample_result.values
        if sample_result.sum():
            result.append(1)
        else:
            result.append(0)
    return result

In [44]:
def disk_FDR(x_test,y_pred,y_test):
    disk_pred=predict_disk(x_test,y_pred,x_test['serial_number'].unique())
    true_state=disk_state(x_test,y_test,x_test['serial_number'].unique())
    true_state=np.array(true_state)
    disk_pred=np.array(disk_pred)
    diskfdr=FDR(true_state, disk_pred)
    return diskfdr

In [45]:
disk_FDR(x_test,y_pred,y_test)

0.917958334554


0.9179583345541914

In [46]:
def disk_FAR(x_test,y_pred,y_test):
    disk_pred=predict_disk(x_test,y_pred,x_test['serial_number'].unique())
    true_state=disk_state(x_test,y_test,x_test['serial_number'].unique())
    true_state=np.array(true_state)
    disk_pred=np.array(disk_pred)
    diskfdr=FAR(true_state, disk_pred)
    return diskfdr

In [47]:
disk_FAR(x_test,y_pred,y_test)

0.36513157894736842

In [16]:
new_list=pd.read_csv('/Users/zhilinyang/Desktop/work/new_list.csv')
new_list=new_list.iloc[:,1]
new_list.head()

0    S300XNYV
1    W300SPHE
2    Z305D6HR
3    Z3029FM3
4    W300BH1H
Name: S300XPAS, dtype: object

In [21]:
len(new_list)

3412

In [19]:
fail_list=pd.read_csv('/Users/zhilinyang/Desktop/work/failed_samples.csv')
fail_list=fail_list['serial_number'].unique()
fail_list

array(['Z300KMM3', 'Z302DK2R', 'Z304H3GQ', ..., 'Z304919P', 'Z30671D8',
       'Z3047S8J'], dtype=object)

In [20]:
len(fail_list)

1058

In [23]:
new_list=new_list.append(pd.Series(fail_list))
len(new_list)

4470

In [30]:
def downsampling(dataset,y,name_list):
    dataset=pd.concat([dataset,y],axis=1)
    dataset=dataset[dataset['serial_number'].isin(name_list)]
    print(len(dataset))
    return dataset   

In [39]:
train=downsampling(x_train,y_train,new_list)
test=downsampling(x_test,y_test,new_list)

19008
13884


In [40]:
new_x_train=train.iloc[:,:-1]
new_x_test=test.iloc[:,:-1]
new_x_test.head()

,serial_number,smart_1_raw,smart_3_normalized,smart_5_normalized,smart_7_normalized,smart_9_normalized,smart_187_normalized,smart_189_normalized,smart_194_normalized,smart_197_normalized,smart_5_raw,smart_197_raw
2,Z301JZLE,235594800.0,93.0,100.0,84.0,90.0,100.0,100.0,22.0,100.0,0.0,0.0
26,Z302ENK8,129171640.0,91.0,100.0,87.0,82.0,100.0,100.0,25.0,100.0,0.0,0.0
28,Z300X9B0,58641392.0,91.0,100.0,81.0,61.0,100.0,89.0,25.0,100.0,0.0,0.0
39,Z300GZD3,4671680.0,91.0,100.0,82.0,59.0,100.0,98.0,25.0,100.0,0.0,0.0
48,W300A1HY,91756984.0,91.0,100.0,84.0,56.0,100.0,95.0,26.0,100.0,0.0,0.0


In [41]:
new_y_train=train.iloc[:,-1]
new_y_test=test.iloc[:,-1]
new_y_test

2         1
26        0
28        0
39        1
48        0
68        0
69        1
85        0
86        0
88        1
89        0
93        0
116       0
120       1
144       0
148       0
173       1
181       0
203       0
204       0
208       0
212       1
216       0
221       0
236       0
240       0
241       0
249       0
251       1
253       0
         ..
105822    0
105835    0
105838    1
105843    0
105862    0
105870    0
105871    0
105879    1
105887    1
105890    0
105914    0
105918    0
105922    0
105933    1
105946    0
105947    0
105956    0
105961    0
105965    0
105969    0
105977    0
105980    0
105991    1
106008    1
106015    0
106016    1
106021    1
106023    0
106030    0
106033    1
Name: 1, Length: 13884, dtype: int64

In [15]:
DT.fit(new_x_train.iloc[:,1:],new_y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.001, min_impurity_split=None,
            min_samples_leaf=7, min_samples_split=20,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [16]:
new_y_pred=DT.predict(new_x_test.iloc[:,1:])

In [18]:
from sklearn.metrics import precision_score
print('precision: %.3f' % precision_score(y_true=new_y_test.values,y_pred=new_y_pred))

precision: 0.466


In [48]:
new_x_train.to_csv('/Users/zhilinyang/Desktop/work/new_x_train.csv')
print('done')

done


In [49]:
new_y_train.to_csv('/Users/zhilinyang/Desktop/work/new_y_train.csv')
print('done')

done


In [50]:
new_x_test.to_csv('/Users/zhilinyang/Desktop/work/new_x_test.csv')
print('done')

done


In [51]:
new_y_test.to_csv('/Users/zhilinyang/Desktop/work/new_y_test.csv')
print('done')

done


In [3]:
new_x_train=pd.read_csv('/Users/zhilinyang/Desktop/work/new_x_train.csv')
new_x_train=new_x_train.iloc[:,1:]
new_x_train.head()

,serial_number,smart_1_raw,smart_3_normalized,smart_5_normalized,smart_7_normalized,smart_9_normalized,smart_187_normalized,smart_189_normalized,smart_194_normalized,smart_197_normalized,smart_5_raw,smart_197_raw
0,Z304HVAM,74048288.0,95.0,100.0,84.0,89.0,100.0,100.0,26.0,100.0,0.0,0.0
1,Z305DJ1J,126636544.0,99.0,100.0,78.0,96.0,100.0,100.0,24.0,100.0,0.0,0.0
2,S300Z567,95737576.0,93.0,100.0,89.0,80.0,100.0,100.0,36.0,100.0,0.0,0.0
3,Z302B0H5,159956032.0,92.0,100.0,89.0,76.0,100.0,100.0,21.0,100.0,0.0,0.0
4,Z305D4MM,140357160.0,92.0,100.0,87.0,84.0,100.0,100.0,35.0,100.0,0.0,0.0


In [4]:
new_y_train=pd.read_csv('/Users/zhilinyang/Desktop/work/new_y_train.csv',header=None)
new_y_train=new_y_train.iloc[:,1]
new_y_train.head()

0    0
1    0
2    0
3    0
4    0
Name: 1, dtype: int64

In [8]:
(len(new_y_train)-8772)/8772

1.1668946648426812

In [9]:
new_x_test=pd.read_csv('/Users/zhilinyang/Desktop/work/new_x_test.csv')
new_x_test=new_x_test.iloc[:,1:]
new_x_test.head()

,serial_number,smart_1_raw,smart_3_normalized,smart_5_normalized,smart_7_normalized,smart_9_normalized,smart_187_normalized,smart_189_normalized,smart_194_normalized,smart_197_normalized,smart_5_raw,smart_197_raw
0,Z301JZLE,235594800.0,93.0,100.0,84.0,90.0,100.0,100.0,22.0,100.0,0.0,0.0
1,Z302ENK8,129171640.0,91.0,100.0,87.0,82.0,100.0,100.0,25.0,100.0,0.0,0.0
2,Z300X9B0,58641392.0,91.0,100.0,81.0,61.0,100.0,89.0,25.0,100.0,0.0,0.0
3,Z300GZD3,4671680.0,91.0,100.0,82.0,59.0,100.0,98.0,25.0,100.0,0.0,0.0
4,W300A1HY,91756984.0,91.0,100.0,84.0,56.0,100.0,95.0,26.0,100.0,0.0,0.0


In [10]:
new_y_test=pd.read_csv('/Users/zhilinyang/Desktop/work/new_y_test.csv',header=None)
new_y_test=new_y_test.iloc[:,1:]
new_y_test.head()

,1
0,1
1,0
2,0
3,1
4,0


In [13]:
(len(new_y_test)-3648)/3648

2.8059210526315788

In [7]:
from sklearn.tree import DecisionTreeClassifier
DT=DecisionTreeClassifier(criterion='entropy',min_samples_split = 20, min_samples_leaf = 7, min_impurity_decrease = 0.001)

In [8]:
DT.fit(new_x_train.iloc[:,1:],new_y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.001, min_impurity_split=None,
            min_samples_leaf=7, min_samples_split=20,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [9]:
new_y_pred=DT.predict(new_x_test.iloc[:,1:])

In [15]:
disk_pred=list()
disk_pred=predict_disk(new_x_test,new_y_pred,new_x_test['serial_number'].unique())

0        0
735      0
1346     0
1968     0
2851     0
4973     0
5391     0
7060     0
7263     0
7913     0
11661    0
12729    0
dtype: int64
3        1
299      1
1651     1
4032     1
4339     1
5555     1
6041     1
10481    1
10871    1
11365    1
12040    1
13022    1
dtype: int64
6        1
361      1
1397     1
1925     1
4067     1
5595     1
6318     1
8351     1
8579     1
10318    1
12612    1
13184    1
dtype: int64
9        0
4994     0
5263     0
5740     0
7733     0
8928     0
10290    1
10660    0
10863    0
10919    0
11013    0
13554    0
dtype: int64
13       0
364      1
748      1
1115     0
2154     1
2342     1
3429     1
4504     1
5164     1
6345     1
9881     1
10084    1
dtype: int64
16       0
1852     0
4044     0
4934     0
7113     0
7315     0
8289     0
8361     0
8621     0
11109    0
11223    0
13379    0
dtype: int64
21       0
429      0
1244     0
3158     0
6559     0
7469     0
7835     0
8863     0
10532    0
11553    0
12935    0
13847    

416      1
1720     1
2674     1
4888     1
7735     1
7911     1
8094     1
8274     1
10060    1
10527    1
11700    1
12385    1
dtype: int64
417      1
487      1
1689     1
5664     1
6701     1
7082     1
7974     1
8584     0
9746     0
10802    1
11495    1
12808    1
dtype: int64
422      1
1551     1
1807     1
1812     1
3020     1
5329     1
5769     1
5988     1
10357    1
10905    1
11925    1
12750    1
dtype: int64
428      1
3235     1
5778     1
6826     1
9493     1
10052    1
10483    1
10540    1
12163    1
12597    1
12623    1
13370    1
dtype: int64
432      0
915      0
1131     0
2977     0
3513     0
4058     0
4456     1
4643     1
8243     1
8775     0
10785    0
13536    0
dtype: int64
440      1
507      0
586      1
1393     1
2750     0
3367     1
3469     1
5614     0
7116     0
7619     0
10945    1
13147    1
dtype: int64
451      1
1264     0
3344     0
5477     0
5749     0
6076     0
6227     0
7778     0
8669     0
10242    0
12721    1
12895    

1297     1
1489     0
3354     0
4769     0
5009     0
5121     0
5723     0
6023     1
9031     0
9057     1
9786     0
13098    0
dtype: int64
1302     1
1564     1
2896     1
3653     1
4111     1
4368     1
6953     1
8831     1
9355     1
11502    1
12132    1
12874    1
dtype: int64
1311     0
2591     0
3570     0
4563     0
5230     0
5433     0
6913     0
8614     0
9833     0
11055    0
13114    0
13485    0
dtype: int64
1321     1
2841     1
4580     1
5126     1
7695     1
8167     1
8341     1
11468    1
11928    1
13504    1
13683    1
13756    1
dtype: int64
1328     0
2535     0
2742     0
2875     0
4019     0
6514     1
6674     0
6949     0
7287     0
7929     0
13195    0
13237    0
dtype: int64
1336     1
2656     1
3104     1
4787     1
5323     1
9424     1
9466     1
11270    1
11633    1
11759    1
13744    1
13845    1
dtype: int64
1407     0
2600     0
3964     0
4027     0
4545     0
5059     0
5556     0
7451     0
9003     0
9211     0
11186    0
13856    

In [13]:
from sklearn.metrics import confusion_matrix
mat=confusion_matrix(y_true=new_y_test.values,y_pred=new_y_pred)
print(mat)

[[7500 2736]
 [1260 2388]]


In [125]:
true_state=list()
true_state=disk_state(new_x_test,new_y_test,new_x_test['serial_number'].unique())

In [127]:
FDR(disk_pred,true_state)

FDR: 0.151


In [23]:
FDR(new_y_test.values,new_y_pred)

0.732708089097


0.73270808909730367

In [49]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [50]:
model = Sequential([
    Dense(11, input_dim=11),
    Activation('relu'),
    Dense(20),
    Activation('relu'),
    Dense(1),
    Activation('softmax'),
])

In [62]:
rmsprop = RMSprop(lr=0.01, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [63]:
model.fit(x_train.iloc[:,1:], y_train, nb_epoch=4, batch_size=100)

/Users/zhilinyang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/4
111159/111159 [==============================] - 1s 13us/step - loss: 14.6843 - acc: 0.0789
Epoch 2/4
111159/111159 [==============================] - 1s 10us/step - loss: 14.6843 - acc: 0.0789
Epoch 3/4
111159/111159 [==============================] - 1s 10us/step - loss: 14.6843 - acc: 0.0789
Epoch 4/4
111159/111159 [==============================] - 1s 11us/step - loss: 14.6843 - acc: 0.0789


In [64]:
loss, accuracy = model.evaluate(x_test.iloc[:,1:], y_test)

print('test loss: ', loss)
print('test accuracy: ', accuracy)

106035/106035 [==============================] - 1s 12us/step
test loss:  15.3939072124
test accuracy:  0.0344037346148
